La primera regla del club del cine es que no se habla del club del cine: queremos utilizar las reviews de las películas para emparejar personas cinéfilas. Dadas las descripciones de las películas que le gustaron (calificación >3) a un usuario y las que no le gustaron (calificación <3) construya dos vectores con tf-idf. Busque para cada usuario, utilizando la concatenación de estos vectores, al usuario más similar. No utilice los usuarios que tienen menos de 30 reviews, ya que no son verdaderos fans del cine. ¿De qué tamaño es el conjunto conexo y poligámico más grande? ¿Quién es el usuario con más parejas?

In [ ]:
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') #Capaz antes de usar nltk les pide que descarguen estas cositas

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
movies = pd.read_csv('/content/drive/MyDrive/Movie dataset/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Movie dataset/ratings.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
ratings = ratings[['userId','movieId','rating']]

In [ ]:
#Filtro a los usuarios
usuariosValidos = ratings.groupby('userId').count()
usuariosValidos = usuariosValidos[usuariosValidos['rating']>150].reset_index() #ACoto a 100 porque si no se me crasheaba por la ram
listUsuariosValidos = list(usuariosValidos['userId'])
usuariosValidos = ratings[ratings['userId'].isin(listUsuariosValidos)]
usuariosValidos

,userId,movieId,rating
414,11,32,3.5
415,11,34,4.0
416,11,47,3.5
417,11,110,3.5
418,11,165,3.5
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


In [ ]:
#Descarto las peliculas que tienen mal el formato
movies = movies[~movies['id'].str.contains('-')]

In [ ]:
#Me quedo con las movies que tiene una descripccion valida y renombro columnas
movies = movies[movies['overview'].notna()]
movies = movies[['id','overview']]
movies = movies.rename(columns={'id':'movieId'})

In [ ]:
movies['movieId'] = movies['movieId'].astype(int)

In [ ]:
#Junto los comentarios con las reviews
calificaciones = pd.merge(usuariosValidos,movies)
calificaciones

,userId,movieId,rating,overview
0,11,110,3.5,Red This is the third film from the trilogy by...
1,24,110,5.0,Red This is the third film from the trilogy by...
2,34,110,5.0,Red This is the third film from the trilogy by...
3,49,110,4.0,Red This is the third film from the trilogy by...
4,56,110,4.0,Red This is the third film from the trilogy by...
...,...,...,...,...
7149660,270123,137072,2.0,Set in the immediate and random background of ...
7149661,270123,140222,3.0,When a young soldier in Vietnam gets dumped by...
7149662,270123,151509,3.0,"The long awaited sequel to R. Kelly's ""Trapped..."
7149663,270123,159109,1.5,"After being released from prison, a man known ..."


In [ ]:
#Separo las descripciones en negativas y positivas segun haya sido la clasificacion
calificaciones['Positivas'] = calificaciones[calificaciones['rating']>3]['overview']
calificaciones['Negativas'] = calificaciones[calificaciones['rating']<3]['overview']
calificaciones = calificaciones.fillna("")
calificaciones = calificaciones[['userId','rating','Negativas','Positivas']]
calificaciones

,userId,rating,Negativas,Positivas
0,11,3.5,,Red This is the third film from the trilogy by...
1,24,5.0,,Red This is the third film from the trilogy by...
2,34,5.0,,Red This is the third film from the trilogy by...
3,49,4.0,,Red This is the third film from the trilogy by...
4,56,4.0,,Red This is the third film from the trilogy by...
...,...,...,...,...
7149660,270123,2.0,Set in the immediate and random background of ...,
7149661,270123,3.0,,
7149662,270123,3.0,,
7149663,270123,1.5,"After being released from prison, a man known ...",


In [ ]:
#Ordeno por usuario mi DataFrame
calificaciones = calificaciones.sort_values('userId',ascending=True)
calificaciones

,userId,rating,Negativas,Positivas
0,11,3.5,,Red This is the third film from the trilogy by...
728007,11,4.0,,The story of a seemingly settled bank employee...
349812,11,2.0,The Czech Republic Director Jan Sverák tells i...,
28052,11,3.5,,Marty and Doc are at it again in this wacky se...
359401,11,4.0,,Threats from sinister foreign nationals aren't...
...,...,...,...,...
2933970,270896,3.5,,Teenagers in a small town are dropping like fl...
788826,270896,4.0,,Lee Yi Min stars as an eager young kung fu stu...
274815,270896,3.5,,A botched robbery indicates a police informant...
1916149,270896,4.5,,When a CIA operation to purchase classified Ru...


In [ ]:
#Agrupo por usuario las descripciones con un join
calificaciones = calificaciones.groupby(['userId'], as_index=False).agg({'Positivas': ' '.join,'Negativas': ' '.join})
calificaciones

,userId,Positivas,Negativas
0,11,Red This is the third film from the trilogy by...,The Czech Republic Director Jan Sverák tells...
1,12,A young Hutterite boy must marry his late brot...,Gilbert has to care for hi...
2,15,In a desperate attempt to win a basketball mat...,A couple of high school graduates spend o...
3,16,Howard Spence has seen better days. Once a big...,...
4,20,Two escaped cons only prayer to escape is to p...,...
...,...,...,...
43720,270872,"When a woman's father goes missing, she enlist...",A budding romance between noble underachieve...
43721,270879,A retired San Francisco detective suffering f...,Mike Sullivan works as a hit man for crime b...
43722,270887,Two tigers are separated as cubs and taken in...,Our hero Zatoichi sails to one of the di...
43723,270893,"Will Penny, an aging cowpoke, takes a job on ...",No overview found. Predominantly set dur...


In [ ]:
#Uso stopwords de ambos idiomas para asegurarme 
from nltk.corpus import stopwords
stopwordsEng = set(stopwords.words('english'))

In [ ]:
#Creo mi countvectorizer con 500 futures para que me de la Ram (en total seran 1000 caundo junte las dos matrices)
countIdf = TfidfVectorizer(lowercase=True, stop_words=stopwordsEng, max_features=500)

In [ ]:
#Creo mi matriz de calificaciones positivas
matrizVectoresIDFP = countIdf.fit_transform(calificaciones['Positivas'])
matrizVectoresIDFP

<43725x500 sparse matrix of type '<class 'numpy.float64'>'
	with 16703272 stored elements in Compressed Sparse Row format>

In [ ]:
#Creo mi matriz de calificaciones negativas
matrizVectoresIDFN = countIdf.fit_transform(calificaciones['Negativas'])
matrizVectoresIDFN

<43725x500 sparse matrix of type '<class 'numpy.float64'>'
	with 7676588 stored elements in Compressed Sparse Row format>

In [ ]:
#Uso el hstack para concatenar horizontalmente mis dos matrices
from scipy.sparse import hstack
concatenada = hstack((matrizVectoresIDFP, matrizVectoresIDFN))

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
#Entreno al buscavecinos como mi matriz ya concatenada
buscaVecinosIdf = NearestNeighbors(metric='cosine', algorithm='brute')
buscaVecinosIdf.fit(concatenada)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
#Busco los vecinos mas cercanos. Al poner None me devuelve el mas cercano de cada fila con la cual entremos al busca vecinos
distancia,indice = buscaVecinosIdf.kneighbors(None)

In [ ]:
#Por cada lista de vecinos encontradas, que tendra el orden de los usuarios ordenados de menor a mayor me voy creando una lista.
matches = []
for index in range(len(indice)):
  usuario = calificaciones['userId'][indice[index][0]] #Obtengo a que usuario corresponde ese indice 
  matches.append(usuario) #Agrego ese usuario a la lista

calificaciones["matches"] = matches #Se lo agrego a una columna a calificaciones que tiene los usuarios agrupados en el mismo orden que estaba la matriz con la cual busco los vecinos
calificaciones 

,userId,Positivas,Negativas,matches
0,11,Red This is the third film from the trilogy by...,The Czech Republic Director Jan Sverák tells...,188500
1,12,A young Hutterite boy must marry his late brot...,Gilbert has to care for hi...,196541
2,15,In a desperate attempt to win a basketball mat...,A couple of high school graduates spend o...,165413
3,16,Howard Spence has seen better days. Once a big...,...,225521
4,20,Two escaped cons only prayer to escape is to p...,...,170470
...,...,...,...,...
43720,270872,"When a woman's father goes missing, she enlist...",A budding romance between noble underachieve...,80599
43721,270879,A retired San Francisco detective suffering f...,Mike Sullivan works as a hit man for crime b...,270123
43722,270887,Two tigers are separated as cubs and taken in...,Our hero Zatoichi sails to one of the di...,270123
43723,270893,"Will Penny, an aging cowpoke, takes a job on ...",No overview found. Predominantly set dur...,3437


In [ ]:
usuarioConMasPareja = calificaciones.groupby("matches").count().sort_values("userId",ascending=False).reset_index().head(1)['matches'][0]
usuarioConMasPareja

179792